In [91]:
import io
import pandas as pd
import requests

In [92]:
# Read the Uber data from the specified file
df = pd.read_csv("D:/Education/data/Uber/uber_data.csv")

In [93]:
# Display information about the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               100000 non-null  int64  
 1   tpep_pickup_datetime   100000 non-null  object 
 2   tpep_dropoff_datetime  100000 non-null  object 
 3   passenger_count        100000 non-null  int64  
 4   trip_distance          100000 non-null  float64
 5   pickup_longitude       100000 non-null  float64
 6   pickup_latitude        100000 non-null  float64
 7   RatecodeID             100000 non-null  int64  
 8   store_and_fwd_flag     100000 non-null  object 
 9   dropoff_longitude      100000 non-null  float64
 10  dropoff_latitude       100000 non-null  float64
 11  payment_type           100000 non-null  int64  
 12  fare_amount            100000 non-null  float64
 13  extra                  100000 non-null  float64
 14  mta_tax                100000 non-nul

In [94]:
# Convert the 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns to datetime format
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int64         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int64         
 4   trip_distance          100000 non-null  float64       
 5   pickup_longitude       100000 non-null  float64       
 6   pickup_latitude        100000 non-null  float64       
 7   RatecodeID             100000 non-null  int64         
 8   store_and_fwd_flag     100000 non-null  object        
 9   dropoff_longitude      100000 non-null  float64       
 10  dropoff_latitude       100000 non-null  float64       
 11  payment_type           100000 non-null  int64         
 12  fare_amount            100000 non-null  float

In [96]:
# Drop duplicate rows from the DataFrame and reset the index
df = df.drop_duplicates().reset_index(drop=True)

# Add a 'trip_id' column based on the index of the DataFrame
df['trip_id'] = df.index

#  Dimension Tables 

## 1) datetime_dim

In [97]:
# Extract the 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns into a new DataFrame called datetime_dim
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)

In [98]:
# Extract various components from the 'tpep_pickup_datetime' column and add them as new columns to datetime_dim
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

In [99]:
# Extract various components from the 'tpep_dropoff_datetime' column and add them as new columns to datetime_dim
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday


In [100]:
# Add a new 'datetime_id' column based on the index of datetime_dim
datetime_dim['datetime_id'] = datetime_dim.index

# Select and reorder the columns of datetime_dim to the desired order
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

# Display the first few rows of datetime_dim
datetime_dim.head()



,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2016-03-01,0,1,3,2016,1,2016-03-01 00:07:55,0,1,3,2016,1
1,1,2016-03-01,0,1,3,2016,1,2016-03-01 00:11:06,0,1,3,2016,1
2,2,2016-03-01,0,1,3,2016,1,2016-03-01 00:31:06,0,1,3,2016,1
3,3,2016-03-01,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1
4,4,2016-03-01,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1


## 2) pickup_location_dim

In [101]:
# Extract the pickup location coordinates into a new DataFrame called pickup_location_dim
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)

# Add a 'pickup_location_id' column based on the index of pickup_location_dim
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index

# Select and reorder the columns of pickup_location_dim to the desired order
pickup_location_dim = pickup_location_dim[['pickup_location_id', 'pickup_latitude', 'pickup_longitude']]


pickup_location_dim.head()

,pickup_location_id,pickup_latitude,pickup_longitude
0,0,40.765152,-73.976746
1,1,40.767925,-73.983482
2,2,40.644810,-73.782021
3,3,40.769814,-73.863419
4,4,40.792183,-73.971741


## 3) dropoff_location_dim

In [102]:
# Extract the dropoff location coordinates into a new DataFrame called dropoff_location_dim
dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)

# Add a 'dropoff_location_id' column based on the index of dropoff_location_dim
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index

# Select and reorder the columns of dropoff_location_dim to the desired order
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id', 'dropoff_latitude', 'dropoff_longitude']]


dropoff_location_dim.head()

,dropoff_location_id,dropoff_latitude,dropoff_longitude
0,0,40.746128,-74.004265
1,1,40.733166,-74.005943
2,2,40.675770,-73.974541
3,3,40.757767,-73.969650
4,4,40.695053,-74.177170


## 4) passenger_count_dim

In [103]:
# Extract the 'passenger_count' column into a new DataFrame called passenger_count_dim
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)

# Add a 'passenger_count_id' column based on the index of the DataFrame
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index

# Select and reorder the columns of passenger_count_dim to the desired order
passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]


passenger_count_dim.head()

,passenger_count_id,passenger_count
0,0,1
1,1,1
2,2,2
3,3,3
4,4,5


## 5) trip_distance_dim

In [104]:
# Extract the 'trip_distance' column into a new DataFrame called trip_distance_dim
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)

# Add a 'trip_distance_id' column based on the index of the DataFrame
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index

# Select and reorder the columns of trip_distance_dim to the desired order
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]


trip_distance_dim.head()

,trip_distance_id,trip_distance
0,0,2.50
1,1,2.90
2,2,19.98
3,3,10.78
4,4,30.43


## 6) payment_type_dim

In [105]:
# Dictionary mapping payment type IDs to their corresponding names
payment_type_name = {
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}

# Extract the 'payment_type' column into a new DataFrame called payment_type_dim
payment_type_dim = df[['payment_type']].reset_index(drop=True)

# Add a 'payment_type_id' column based on the index of the DataFrame
payment_type_dim['payment_type_id'] = payment_type_dim.index

# Map the payment type IDs to their corresponding names using the payment_type_name dictionary
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)

# Select and reorder the columns of payment_type_dim to the desired order
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]


payment_type_dim.head()

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card


## 7) rate_code_dim

In [106]:
# This dictionary defines the mapping between rate code IDs and their corresponding names.
rate_code_type = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride"
}

# Creating a new DataFrame called 'rate_code_dim' containing the 'RatecodeID' column from the 'df' DataFrame.
# The 'reset_index(drop=True)' method resets the index of the new DataFrame and drops the old index.
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)

# Adding two new columns to the 'rate_code_dim' DataFrame.
# 'rate_code_id' column is created and assigned the index values of the DataFrame.
# 'rate_code_name' column is created and assigned the corresponding rate code names using the mapping defined in 'rate_code_type'.
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)

# Selecting only the desired columns ('rate_code_id', 'RatecodeID', 'rate_code_name') from the 'rate_code_dim' DataFrame.
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]


rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,1,Standard rate
2,2,1,Standard rate
3,3,1,Standard rate
4,4,3,Newark


#  Fact Table

In [107]:
# Merge dimension tables with the main DataFrame to create the fact table.
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]


In [108]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,1,N,1,1,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,2,2,2,N,2,2,2,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,3,3,3,N,3,3,3,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,4,2,4,4,4,4,N,4,4,4,98.0,0.0,0.0,0.00,15.50,0.3,113.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,99995,99995,99995,99995,N,99995,99995,99995,5.0,0.0,0.5,0.00,0.00,0.3,5.80
99996,99996,1,99996,99996,99996,99996,N,99996,99996,99996,14.0,0.0,0.5,2.00,0.00,0.3,16.80
99997,99997,1,99997,99997,99997,99997,N,99997,99997,99997,29.0,0.0,0.5,8.80,5.54,0.3,44.14
99998,99998,2,99998,99998,99998,99998,N,99998,99998,99998,5.5,0.5,0.5,1.36,0.00,0.3,8.16
